# Day 25: Clock Signal

## Part One

You open the door and find yourself on the roof. The city sprawls away from you for miles and miles.

There's not much time now - it's already Christmas, but you're nowhere near the North Pole, much too far to deliver these stars to the sleigh in time.

However, maybe the huge antenna up here can offer a solution. After all, the sleigh doesn't need the stars, exactly; it needs the timing data they provide, and you happen to have a massive signal generator right here.

You connect the stars you have to your prototype computer, connect that to the antenna, and begin the transmission.

Nothing happens.

You call the service number printed on the side of the antenna and quickly explain the situation. "I'm not sure what kind of equipment you have connected over there," he says, "but you need a clock signal." You try to explain that this is a signal for a clock.

"No, no, a clock signal - timing information so the antenna computer knows how to read the data you're sending it. An endless, alternating pattern of `0`, `1`, `0`, `1`, `0`, `1`, `0`, `1`, `0`, `1`...." He trails off.

You ask if the antenna can handle a clock signal at the frequency you would need to use for the data from the stars. "There's no way it can! The only antenna we've installed capable of that is on top of a top-secret Easter Bunny installation, and you're definitely not-" You hang up the phone.

You've extracted the antenna's clock signal generation assembunny code (your puzzle input); it looks mostly compatible with code you worked on just recently.

This antenna code, being a signal generator, uses one extra instruction:

* `out x` transmits x (either an integer or the value of a register) as the next value for the clock signal.

The code takes a value (via register a) that describes the signal to generate, but you're not sure how it's used. You'll have to find the input to produce the right signal through experimentation.

What is the lowest positive integer that can be used to initialize register `a` and cause the code to output a clock signal of `0`, `1`, `0`, `1`... repeating forever?

---

In [1]:
# Initialise
inputs = [i[:-1].split(' ') for i in open('Day25.in').readlines()]

In [2]:
class CPU:
    def __init__(self, register=None, queue=None, out=None):
        if register is None:
            self.register = {char:0 for char in ('a', 'b', 'c', 'd')}
        else:
            self.register = register
            
        self.future = self.register.copy()
        
        if queue is None:
            self.queue = []
        else:
            self.queue = queue
        if out is None:
            self.out = []
        else:
            self.out = out
        
        self.max_range = 8
    
    @staticmethod
    def parse(register, out, cmd, x=None, y=None, z=None):
        def val(z):
            if z in register:
                return register[z]
            else:
                return int(z)
        
        if cmd == 'nop':
            pass
        elif cmd == 'cpy':
            register[y] = val(x)
        elif cmd == 'out':
            if out is not None:
                out.append(val(x))
        elif cmd == 'inc':
            register[x] += 1
        elif cmd == 'dec':
            register[x] -= 1
        elif cmd == 'jnz':
            pass
        elif cmd == 'tgl':
            pass
        elif cmd == 'add':
            # Create add functionality
            # <add x y> adds x to register y
            register[y] += val(x)
        elif cmd == 'sub':
            # Create subtraction functionality
            # <sub x y> subtracts x from register y
            register[y] -= val(x)
        elif cmd == 'mul':
            # Create multiplication functionality
            # <mul x y z> adds x*y to register z
            register[z] += val(x)*val(y)
        elif cmd == 'flr':
            # Create floor division functionality
            # <flr x y z> adds x//y to register z
            register[z] += val(x)//val(y)
    
    def compute(self, cmd, x=None, y=None, z=None):
        self.parse(self.register, self.out, cmd, x, y, z)
    
    def predict(self, cmd, x=None, y=None, z=None):
        self.parse(self.future, None, cmd, x, y, z)
    
    def step(self):
        self.compute(*self.queue[0])
        self.queue = self.queue[1:]
    
    def resolve(self, n=None):
        if n is None:
            for task in self.queue:
                self.compute(*task)
            self.queue = []
        else:
            for task in self.queue[:n]:
                self.compute(*task)
            self.queue = self.queue[n:]
    
    def refresh(self):
        self.future = self.register.copy()
        for task in self.queue:
            self.predict(*task)
    
    def add(self, task):
        self.queue.append(task)
        self.predict(*task)
        if len(self) > self.max_range:
            self.step()
    
    def __len__(self):
        return len(self.queue)
    
    def peephole(self):
        """
        See if it's possible to optimise the last jnz added to the queue.
        
        Warning: peephole may resolve part of the queue to optmise things correctly
        """
        # translates things into add, length 3
        if (self.queue[-1][0] == 'jnz') and ((k := int(self.queue[-1][2])) == -2):
            jnz_char = self.queue[-1][1]
            if self.queue[k-1][0] == 'inc':
                add_char = self.queue[k-1][1]
                if (self.queue[k][0] == 'dec') and (self.queue[k][1] == jnz_char):
                    temp = self.queue[:k-1]
                    temp += [['add', jnz_char, add_char], ['cpy', '0', jnz_char], ['nop', '0']]
                    self.queue = temp
                    self.refresh()
                    return True
        
        # translates things into a mul, length 6
        if (self.queue[-1][0] == 'jnz') and ((k := int(self.queue[-1][2])) == -5):
            jnz_char = self.queue[-1][1]
            if self.queue[k-1][0] == 'cpy':
                m, mul_char = self.queue[k-1][1:]
                if self.queue[k][0:2] == ['add', mul_char]:
                    reg_char = self.queue[k][2]
                    if self.queue[k+3] == ['dec', jnz_char]:
                        temp = self.queue[:k-1]
                        temp += [
                            ['mul', m, jnz_char, reg_char], 
                            ['cpy', '0', jnz_char], 
                            ['cpy', '0', mul_char],
                            ['nop', '0'],
                            ['nop', '0'],
                            ['nop', '0']
                        ]
                        self.queue = temp
                        self.refresh()
                        return True
                        
        return False
    
    def __str__(self):
        s = "\n  ".join([str(t) for t in self.queue])
        return f'CPU' + str(self.register) + '[' + ('\n  ' if s else '') + s + ('\n' if s else '') + ']' + str(self.future)
    
    def __repr__(self):
        return str(self)

                    

In [3]:
n = len(inputs)
a = 0

while True:
    a += 1
    cursor = 0
    cpu = CPU(
        register={'a':a, 'b':0, 'c':0, 'd':0}
    )

    future_val = lambda v: cpu.future[v] if v in cpu.future else int(v)
    while (0 <= cursor <= n) and (len(cpu.out) < 10) :
        t = inputs[cursor]
        cpu.add(t)
        if (inputs[cursor][0] == 'jnz') and future_val(inputs[cursor][1]):
            optimised = cpu.peephole()
            cursor += 0 if optimised else int(t[2])-1
        cursor += 1
        if cpu.out != ([0,1]*5)[:len(cpu.out)]:
            break
    
    if cpu.out == [0,1]*5:
        # Solution
        print(f"Setting a={cpu.register['a']} causes the output to be 0, 1, 0, 1,... repeating")
        break

Setting a=2 causes the output to be 0, 1, 0, 1,... repeating


---